In [1]:
%load_ext autoreload
%autoreload 2

# Settings

In [2]:
%env TF_KERAS = 1
import os
sep_local = os.path.sep

import sys
sys.path.append('..'+sep_local+'..')
print(sep_local)

In [3]:
import tensorflow as tf
print(tf.__version__)

2.3.0


In [4]:
os.chdir('..'+sep_local+'..'+sep_local+'..'+sep_local+'..'+sep_local+'..')
print(os.getcwd())

C:\Users\Khalid\Documents\projects\Generative_Models


# only on colab

In [5]:
#!cat data/.pokemon/pokemon.tar.part* > data/.pokemon/pokemon.tar

In [6]:
#!tar -xvf data/.pokemon/pokemon_combined.tar --directory data/.pokemon/

# Dataset loading

In [7]:
from training.generators.from_lmdb.lmdb_image_generator import get_generators

Using TensorFlow backend.


In [8]:
dataset_name='pokemon'
inputs_shape= image_shape=(100, 100, 3)
batch_size = 10
latents_dim = 32
intermediate_dim = 50
lmdb_dir = 'data/.pokemon/Pokemon_LMDB'

In [9]:
from transformation.lmdb_transformer import LmdbTransformer

In [10]:
train_generator, val_generator = get_generators(
        lmdb_dir=lmdb_dir,
        batch_size=batch_size,
        episode_len=None,
        episode_shift=None
)

  DEBUG    | Looking for images in '_training'
  DEBUG    | Looking for images in '_training'


  WARNING  | No files found
  WARNING  | No files found
C:\Users\Khalid\Documents\projects\Generative_Models\transformation\file_image_generator.py:51: UserWarning: No files found
  warnings.warn(msg)


  DEBUG    | Looking for images in '_validation'
  DEBUG    | Looking for images in '_validation'


  WARNING  | No files found
  WARNING  | No files found


Initializing Iterator training Number of images 591
Initializing Iterator validation Number of images 218


In [11]:
_instance_scale=1.0
for data in val_generator:
    print(data['images'].numpy().max())
    break
    #print(data.keys())

1.0


In [12]:
data['images'][0].shape

TensorShape([100, 100, 3])

In [13]:
data.keys()

dict_keys(['images', 'label'])

# Model's Layers definition

In [14]:
import numpy as np
from collections.abc import Iterable
if isinstance(inputs_shape, Iterable):
    flat_outputs_shape = np.prod(inputs_shape)
flat_outputs_shape

30000

In [15]:
latents_dim

32

In [16]:
enc_lays = [tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Dense(units=intermediate_dim, activation='relu'),
            tf.keras.layers.Flatten(),
            tf.keras.layers.Dense(units=latents_dim)]

dec_lays = [tf.keras.layers.Dense(units=latents_dim, activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),
            tf.keras.layers.Dense(units= latents_dim*intermediate_dim//3 , activation='relu'),

            tf.keras.layers.Dense(units=flat_outputs_shape),
            tf.keras.layers.Reshape(inputs_shape)]

# Model definition

In [17]:
model_name = dataset_name+'AE_Dense_reconst_ell'
experiments_dir='experiments'+sep_local+model_name

In [18]:
from training.autoencoding_basic.autoencoders.autoencoder import autoencoder as AE

In [19]:
variables_params = \
[
    {
        'name': 'inference', 
        'inputs_shape':inputs_shape,
        'outputs_shape':latents_dim,
        'layers': enc_lays
    }

    ,
    
        {
        'name': 'generative', 
        'inputs_shape':latents_dim,
        'outputs_shape':inputs_shape,
        'layers':dec_lays
    }
]

In [20]:
from utils.data_and_files.file_utils import create_if_not_exist

In [21]:
_restore = os.path.join(experiments_dir, 'var_save_dir')

In [22]:
create_if_not_exist(_restore)
_restore

'experiments\\pokemonAE_Dense_reconst_ell\\var_save_dir'

In [23]:
#to restore trained model, set filepath=_restore

In [24]:
ae = AE( 
    name=model_name,
    latents_dim=latents_dim,
    batch_size=batch_size,
    variables_params=variables_params, 
    filepath=None
    )

Model: "inference"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
dense (Dense)                (None, 100, 100, 50)      200       
_________________________________________________________________
dense_1 (Dense)              (None, 100, 100, 50)      2550      
_________________________________________________________________
flatten (Flatten)            (None, 500000)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 32)                16000032  
_________________________________________________________________
batch_normalization (BatchNo (None, 32)                128       
_________________________________________________________________
dropout (Dropout)            (None, 32)                0 

  WARNING  | None
  WARNING  | None


Model: "generative"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
generative_inputs (InputLaye [(None, 32)]              0         
_________________________________________________________________
dense_3 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_4 (Dense)              (None, 533)               17589     
_________________________________________________________________
dense_5 (Dense)              (None, 533)               284622    
_________________________________________________________________
dense_6 (Dense)              (None, 533)               284622    
_________________________________________________________________
dense_7 (Dense)              (None, 30000)             16020000  
_________________________________________________________________
reshape (Reshape)            (None, 100, 100, 3)       0

  WARNING  | None
  WARNING  | None


In [25]:
#ae.compile(metrics=None)
ae.compile()

Model: "pokemonAE_Dense_reconst_ell"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inference_inputs (InputLayer [(None, 100, 100, 3)]     0         
_________________________________________________________________
inference (Functional)       (None, 32)                16002910  
_________________________________________________________________
generative (Functional)      (None, 100, 100, 3)       16607901  
_________________________________________________________________
tf_op_layer_x_logits (Tensor [(None, 100, 100, 3)]     0         
Total params: 32,610,811
Trainable params: 32,610,741
Non-trainable params: 70
_________________________________________________________________
None


In [26]:
ae.output_shape

(None, 100, 100, 3)

# Callbacks

In [27]:
from training.callbacks.sample_generation import SampleGeneration
from training.callbacks.save_model import ModelSaver

In [28]:
es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    min_delta=1e-15, 
    patience=10, 
    verbose=1, 
    restore_best_weights=False
)

In [29]:
ms = ModelSaver(filepath=_restore)

In [30]:
csv_dir = os.path.join(experiments_dir, 'csv_dir')
create_if_not_exist(csv_dir)
csv_dir = os.path.join(csv_dir, ae.name+'.csv')
csv_log = tf.keras.callbacks.CSVLogger(csv_dir, append=True)
csv_dir

'experiments\\pokemonAE_Dense_reconst_ell\\csv_dir\\pokemonAE_Dense_reconst_ell.csv'

In [31]:
image_gen_dir = os.path.join(experiments_dir, 'image_gen_dir')
create_if_not_exist(image_gen_dir)

In [32]:
sg = SampleGeneration(latents_shape=latents_dim, filepath=image_gen_dir, gen_freq=5, save_img=True, gray_plot=False)

In [33]:
import numpy as np

# Model Training

In [34]:
ae.fit(
    x=train_generator,
    input_kw='images',
    steps_per_epoch=int(5e3),
    epochs=int(1e6), 
    verbose=2,
    callbacks=[es, ms, csv_log, sg],
    workers=-1,
    use_multiprocessing=True,
    validation_data=val_generator,
    validation_steps=int(1e4)
)

Epoch 1/1000000

Epoch 00001: loss improved from inf to 7804.29102, saving model to experiments\pokemonAE_Dense_reconst_ell\var_save_dir
5000/5000 - 725s - loss: 7804.2910 - psnr: 10.6674 - total_variation: 82763.9531 - ssim_multiscale: 3.1985 - sharpdiff: 5.3070 - mean_absolute_error: 0.2202 - mean_squared_error: 0.0882 - val_loss: 3111.4226 - val_psnr: 15.6926 - val_total_variation: 24531.2520 - val_ssim_multiscale: 7.0578 - val_sharpdiff: 10.1793 - val_mean_absolute_error: 0.1179 - val_mean_squared_error: 0.0292
Epoch 2/1000000

Epoch 00002: loss improved from 7804.29102 to 5990.32471, saving model to experiments\pokemonAE_Dense_reconst_ell\var_save_dir
5000/5000 - 706s - loss: 5990.3247 - psnr: 11.6867 - total_variation: 91814.7812 - ssim_multiscale: 4.7241 - sharpdiff: 5.1741 - mean_absolute_error: 0.1814 - mean_squared_error: 0.0689 - val_loss: 1852.1803 - val_psnr: 18.6998 - val_total_variation: 29570.5723 - val_ssim_multiscale: 8.2679 - val_sharpdiff: 11.1435 - val_mean_absolut

Epoch 17/1000000

Epoch 00017: loss improved from 4248.61768 to 4230.20752, saving model to experiments\pokemonAE_Dense_reconst_ell\var_save_dir
5000/5000 - 684s - loss: 4230.2075 - psnr: 13.0570 - total_variation: 102077.6016 - ssim_multiscale: 6.5032 - sharpdiff: 5.2421 - mean_absolute_error: 0.1289 - mean_squared_error: 0.0504 - val_loss: 274.7545 - val_psnr: 27.1472 - val_total_variation: 29129.4883 - val_ssim_multiscale: 9.7809 - val_sharpdiff: 15.2911 - val_mean_absolute_error: 0.0251 - val_mean_squared_error: 0.0022
Epoch 18/1000000

Epoch 00018: loss improved from 4230.20752 to 4229.06006, saving model to experiments\pokemonAE_Dense_reconst_ell\var_save_dir
5000/5000 - 685s - loss: 4229.0601 - psnr: 13.0676 - total_variation: 102090.8906 - ssim_multiscale: 6.5191 - sharpdiff: 5.2468 - mean_absolute_error: 0.1284 - mean_squared_error: 0.0503 - val_loss: 6674.1318 - val_psnr: 12.5388 - val_total_variation: 25275.3965 - val_ssim_multiscale: 4.4188 - val_sharpdiff: 9.5313 - val_mea

# Model Evaluation

## inception_score

In [ ]:
from evaluation.generativity_metrics.inception_metrics import inception_score

In [ ]:
is_mean, is_sigma = inception_score(ae, tolerance_threshold=1e-6, max_iteration=100)
print(f'inception_score mean: {is_mean}, sigma: {is_sigma}')

## Frechet_inception_distance

In [ ]:
is_mean.compute()

In [ ]:
from evaluation.generativity_metrics.inception_metrics import frechet_inception_distance

In [ ]:
fis_score = frechet_inception_distance(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'frechet inception distance: {fis_score}')

## perceptual_path_length_score

In [ ]:
from evaluation.generativity_metrics.perceptual_path_length import perceptual_path_length_score

In [ ]:
ppl_mean_score = perceptual_path_length_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100, batch_size=32)
print(f'perceptual path length score: {ppl_mean_score}')

## precision score

In [ ]:
from evaluation.generativity_metrics.precision_recall import precision_score

In [ ]:
_precision_score = precision_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'precision score: {_precision_score}')

## recall score

In [ ]:
from evaluation.generativity_metrics.precision_recall import recall_score

In [ ]:
_recall_score = recall_score(ae, training_generator, tolerance_threshold=1e-6, max_iteration=100)
print(f'recall score: {_recall_score}')

# Image Generation

## image reconstruction

### Training dataset

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from training.generators.image_generation_testing import reconstruct_from_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'reconstruct_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

reconstruct_from_a_batch(ae, testing_generator, save_dir)

## with Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_like_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_training_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, training_generator, save_dir)

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'generate_testing_images_like_a_batch_dir')
create_if_not_exist(save_dir)

generate_images_like_a_batch(ae, testing_generator, save_dir)

### Complete Randomness

In [ ]:
from training.generators.image_generation_testing import generate_images_randomly

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'random_synthetic_dir')
create_if_not_exist(save_dir)

generate_images_randomly(ae, save_dir)

In [ ]:
from training.generators.image_generation_testing import interpolate_a_batch

In [ ]:
from utils.data_and_files.file_utils import create_if_not_exist
save_dir = os.path.join(experiments_dir, 'interpolate_dir')
create_if_not_exist(save_dir)

interpolate_a_batch(ae, testing_generator, save_dir)